In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import numpy as np

def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity

def preprocess_text(text):
    # 특수문자 제거
    text = ''.join(char for char in text if char.isalnum() or char.isspace())

    # 불용어 처리 (한국어)
    okt = Okt()
    tokens = okt.morphs(text)
    stopwords = ['은', '는', '이', '가', '을', '를', '것', '그', '에', '어', '하다']
    tokens = [token for token in tokens if token not in stopwords]

    return " ".join(tokens)

def cosine_similarity_between_preprocessed_korean_sentences(prompt, prepared_sentences):
    # 전처리된 문장
    preprocessed_sentence1 = preprocess_text(prompt)
    preprocessed_prepared_sentences = [ preprocess_text(sentence) for sentence in prepared_sentences ]

    # TF-IDF 벡터화 객체 생성
    vectorizer = TfidfVectorizer()

    # 두 문장을 합쳐서 벡터화
    sentences = [preprocessed_sentence1, *preprocessed_prepared_sentences]
    tfidf_matrix = vectorizer.fit_transform(sentences)
    
    feature_vect_dense = tfidf_matrix.todense()
    
    vect1 = np.array(feature_vect_dense[0]).reshape(-1)
    vect2 = np.array(feature_vect_dense[1]).reshape(-1)
    
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
    cos_sim = cos_similarity(vect1, vect2)

    # 결과 출력
    print(f"전처리된 prompt:\n\t{preprocessed_sentence1}")
    print_text = [ f"{text}\n" for text in preprocessed_prepared_sentences]
    print(f"전처리된 prepared sentences:\n\t{print_text}")
    print(f"코사인 유사도: {cos_sim}")
    print(f"코사인 유사도: {cosine_sim[0][1:]}")

# 예시 문장 (한국어)
prompt = "나 스킨 화장품 하나만 추천해줘"
prepared_sentences = [
    "화장품 하나 추천해줘",
    "민감성 피부에 좋은 화장품 하나 추천해주라",
    "토너 몇개만 추천해줘"
]

# 코사인 유사도 계산 (전처리 포함)
cosine_similarity_between_preprocessed_korean_sentences(prompt, prepared_sentences)


전처리된 prompt:
	나 스킨 화장품 하나 만 추천 해줘
전처리된 prepared sentences:
	['화장품 하나 추천 해줘\n', '민감 성 피부 좋은 화장품 하나 추천 해주라\n', '토너 몇개 만 추천 해줘\n']
코사인 유사도: 0.7740323183879741
코사인 유사도: [0.77403232 0.30517734 0.2629022 ]
